In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import sys
sys.path.append('../../')
from analysts_tools.growth import *

In [2]:
def highlight_max(s):
    '''
    Highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [3]:
fields = ['region_code', "gmv_usd", "cant",'submit_date', 'close_date', 'customer_id', 'category', 'subcat']

In [4]:
#read csv
data_ventas_2020_06_to_2020_09 = pd.read_csv('data_ventas_2020-06-01_2020-09-30.csv', skipinitialspace=True, usecols=fields)
data_ventas_2020_10_to_2021_01 = pd.read_csv('data_ventas_2020-10-01_2021-01-31.csv', skipinitialspace=True, usecols=fields)
data_ventas_2021_02_to_2021_05 = pd.read_csv('data_ventas_2021-02-01_2021-05-31.csv', skipinitialspace=True, usecols=fields)
data_ventas_2021_06_to_2021_09 = pd.read_csv('data_ventas_2021-06-01_2021-09-30.csv', skipinitialspace=True, usecols=fields)
data_ventas_2022_07_to_2022_07 = pd.read_csv('data_ventas_2022-07-01_2022-07-31.csv', skipinitialspace=True, usecols=fields)
data_ventas_2022_08_to_2022_08 = pd.read_csv('data_ventas_2022-08-01_2022-08-31.csv', skipinitialspace=True, usecols=fields)
#big info
data_ventas_2021_10_to_2021_11 = pd.read_csv('data_ventas_2021-10-01_2021-11-30.csv', skipinitialspace=True, usecols=fields)
#data_ventas_2021_10_to_2021_11 = data_ventas_2021_10_to_2021_11.iloc[:,1:].copy()
data_ventas_2021_12_to_2022_01 = pd.read_csv('data_ventas_2021-12-01_2022-01-31.csv', skipinitialspace=True, usecols=fields)
#data_ventas_2021_12_to_2022_01 = data_ventas_2021_12_to_2022_01.iloc[:,1:].copy()
data_ventas_2022_02_to_2022_03 = pd.read_csv('data_ventas_2022-02-01_2022-03-31.csv', skipinitialspace=True, usecols=fields)
#data_ventas_2022_02_to_2022_03 = data_ventas_2022_02_to_2022_03.iloc[:,1:].copy()
data_ventas_2022_04_to_2022_05 = pd.read_csv('data_ventas_2022-04-01_2022-05-31.csv', skipinitialspace=True, usecols=fields)
#data_ventas_2022_04_to_2022_05 = data_ventas_2022_04_to_2022_05.iloc[:,1:].copy()
data_ventas_2022_06_to_2022_06 = pd.read_csv('data_ventas_2022-06-01_2022-06-30.csv', skipinitialspace=True, usecols=fields)
#data_ventas_2022_06_to_2022_06 = data_ventas_2022_06_to_2022_06.iloc[:,1:].copy()

In [5]:
frames = [data_ventas_2020_06_to_2020_09, data_ventas_2020_10_to_2021_01, 
          data_ventas_2021_02_to_2021_05, data_ventas_2021_06_to_2021_09, 
          data_ventas_2021_10_to_2021_11, data_ventas_2021_12_to_2022_01, 
          data_ventas_2022_02_to_2022_03, data_ventas_2022_04_to_2022_05, 
          data_ventas_2022_06_to_2022_06, data_ventas_2022_07_to_2022_07,
          data_ventas_2022_08_to_2022_08]
result = pd.concat(frames)

In [9]:
birthday

,customer_id,birthday
0,-269774,2020-03-12 21:19:43.444219
1,-269770,2019-10-24 17:35:45.210136
2,-269652,2019-10-15 15:14:55.659760
3,-269587,2019-09-02 22:47:00.565060
4,-269416,2019-08-18 22:59:29.785564
...,...,...
183758,201899060,2022-08-10 22:27:07.665000
183759,215773160,2022-09-01 13:27:41.644000
183760,215792647,2022-09-01 13:51:35.779000
183761,204517696,2022-08-10 16:05:06.098000


In [8]:
data_ventas = result
birthday = pd.read_csv('data_birthdays_ALL.csv', usecols=['customer_id','birthday'])


data_ventas["birthday_date"] = data_ventas["customer_id"].map(birthday)
assert (data_ventas["birthday_date"] <= data_ventas["submit_date"]).all()

data_ventas["submit_month"] = pd.to_datetime(data_ventas["submit_date"].dt.strftime("%Y-%m"), format="%Y-%m")
data_ventas["birthday_month"] = pd.to_datetime(data_ventas["birthday_date"].dt.strftime("%Y-%m"), format="%Y-%m")
del(birthday)

data_ventas["months_alive"] = np.round((data_ventas["submit_month"] - data_ventas["birthday_month"]).dt.days/30)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
pivoted_table = pd.pivot_table(data=data_ventas, columns=["months_alive"], index=["birthday_month"], values=["customer_id", "gmv_usd", "order_id"], aggfunc={"customer_id":pd.Series.nunique, "gmv_usd":np.sum, "order_id":pd.Series.nunique})
pivoted_table = pivoted_table[pivoted_table.index >= data_ventas["submit_month"].min()].dropna(axis=1, how="all").copy()
assert pivoted_table["gmv_usd"].fillna(value=0).values.sum()/data_ventas[data_ventas["birthday_month"] >= data_ventas["submit_month"].min()]["gmv_usd"].sum() - 1 < 0.0000004
pivoted_table.index = [str(x.year) + "-" + str(x.month) for x in pivoted_table.index]

print("-----------------#Clientes------------")
display((pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("{:.1f}", na_rep="-"))

print("-----------------GMV------------------")
display((pivoted_table["gmv_usd"]).style.apply(highlight_max, axis=1).format("${:,.3f}", na_rep="-"))

print("-----------------ARPU-----------------")
display((pivoted_table["gmv_usd"]/pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("${:.2f}", na_rep="-"))

print("-----------------Frecuency-----------------")
display((pivoted_table["order_id"]/pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("{:.3}", na_rep="-"))

print("-----------------AoV-----------------")
display((pivoted_table["gmv_usd"]/pivoted_table["order_id"]).style.apply(highlight_max, axis=1).format("${:.2f}", na_rep="-"))

In [ ]:
grouped_df = data_ventas.groupby(["customer_id", "months_alive"]).agg({"birthday_month":pd.Series.unique, "category":pd.Series.nunique, "subcat":pd.Series.nunique}).reset_index()
pivoted_table_2 = pd.pivot_table(data=grouped_df, columns=["months_alive"], index=["birthday_month"], values=["category", "subcat"], aggfunc={"category":np.mean, "subcat":np.mean})
pivoted_table_2 = pivoted_table_2[pivoted_table_2.index >= data_ventas["submit_month"].min()].dropna(axis=1, how="all").copy()
pivoted_table_2.index = [str(x.year) + "-" + str(x.month) for x in pivoted_table_2.index]


print("-----------------Category-----------------")
display(pivoted_table_2["category"].style.apply(highlight_max, axis=1).format("{:.3}", na_rep="-"))

print("-----------------Subcat-----------------")
display(pivoted_table_2["subcat"].style.apply(highlight_max, axis=1).format("{:.3}", na_rep="-"))

In [ ]:
def customer_id_number_of_category_first_month(df):
    a = df[df["months_alive"] == 0][["customer_id", "category"]]
    answer = dict(zip(a["customer_id"], a["category"]))
    return answer

def customer_id_number_of_subcat_first_month(df):
    a = df[df["months_alive"] == 0][["customer_id", "subcat"]]
    answer = dict(zip(a["customer_id"], a["subcat"]))
    return answer
num_of_cat_first_month = customer_id_number_of_category_first_month(df=grouped_df)
num_of_subcat_first_month = customer_id_number_of_subcat_first_month(df=grouped_df)

In [ ]:
data_ventas["category_first_month"] = data_ventas["customer_id"].map(num_of_cat_first_month)
data_ventas["subcat_first_month"] = data_ventas["customer_id"].map(num_of_subcat_first_month)

In [ ]:
# Cambiar el numero para filtrar por n categorias compradas en el mes de nacimiento
otro = data_ventas[data_ventas["category_first_month"] == 1]


pivoted_table = pd.pivot_table(data=otro, columns=["months_alive"], index=["birthday_month"], values=["customer_id", "gmv_usd", "order_id"], aggfunc={"customer_id":pd.Series.nunique, "gmv_usd":np.sum, "order_id":pd.Series.nunique})
pivoted_table = pivoted_table[pivoted_table.index >= otro["submit_month"].min()].dropna(axis=1, how="all").copy()
assert pivoted_table["gmv_usd"].fillna(value=0).values.sum()/otro[otro["birthday_month"] >= otro["submit_month"].min()]["gmv_usd"].sum() - 1 < 0.0000004
pivoted_table.index = [str(x.year) + "-" + str(x.month) for x in pivoted_table.index]

print("-----------------#Clientes------------")
display((pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("{:.1f}", na_rep="-"))

print("-----------------GMV------------------")
display((pivoted_table["gmv_usd"]).style.apply(highlight_max, axis=1).format("${:,.2f}", na_rep="-"))

print("-----------------ARPU-----------------")
display((pivoted_table["gmv_usd"]/pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("${:.2f}", na_rep="-"))

print("-----------------Frecuency-----------------")
display((pivoted_table["order_id"]/pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("{:.2}", na_rep="-"))

print("-----------------AoV-----------------")
display((pivoted_table["gmv_usd"]/pivoted_table["order_id"]).style.apply(highlight_max, axis=1).format("${:.2f}", na_rep="-"))

grouped_df = otro.groupby(["customer_id", "months_alive"]).agg({"birthday_month":pd.Series.unique, "category":pd.Series.nunique, "subcat":pd.Series.nunique}).reset_index()
pivoted_table_2 = pd.pivot_table(data=grouped_df, columns=["months_alive"], index=["birthday_month"], values=["category", "subcat"], aggfunc={"category":np.mean, "subcat":np.mean})
pivoted_table_2 = pivoted_table_2[pivoted_table_2.index >= otro["submit_month"].min()].dropna(axis=1, how="all").copy()
pivoted_table_2.index = [str(x.year) + "-" + str(x.month) for x in pivoted_table_2.index]


print("-----------------Category-----------------")
display(pivoted_table_2["category"].style.apply(highlight_max, axis=1).format("{:.2}", na_rep="-"))

print("-----------------Subcat-----------------")
display(pivoted_table_2["subcat"].style.apply(highlight_max, axis=1).format("{:.2}", na_rep="-"))

In [ ]:
# prueba = data_ventas[["customer_id", "birthday_month", "submit_month", "gmv_usd", "order_id", "category", "subcat", "months_alive"]]
# prueba_grouped = prueba.groupby(by=["customer_id", "months_alive", "birthday_month"]).agg({"gmv_usd":np.sum, "order_id":pd.Series.nunique, "category":pd.Series.nunique, "subcat":pd.Series.nunique}).reset_index()

# prueba_pivot = pd.pivot_table(data=prueba_grouped, values=["order_id", "category", "subcat", "customer_id"], columns=["months_alive"], index=["birthday_month"], aggfunc={"order_id":np.sum, "category":np.mean, "subcat":np.mean, "customer_id":pd.Series.nunique})
# prueba_pivot = prueba_pivot[prueba_pivot.index >= data_ventas["submit_month"].min()].dropna(axis=1, how="all").copy()
# assert (pivoted_table["customer_id"].fillna(0).values == prueba_pivot["customer_id"].fillna(0).values).all()
# assert (pivoted_table["order_id"].fillna(0).values == prueba_pivot["order_id"].fillna(0).values).all()
# assert (prueba_pivot["category"].fillna(0).values == pivoted_table_2["category"].fillna(0).values).all()
# assert (prueba_pivot["subcat"].fillna(0).values == pivoted_table_2["subcat"].fillna(0).values).all()

In [ ]:
# Cambiar el numero para filtrar por n categorias compradas en el mes de nacimiento
otro = data_ventas[data_ventas["category_first_month"] == 2]


pivoted_table = pd.pivot_table(data=otro, columns=["months_alive"], index=["birthday_month"], values=["customer_id", "gmv_usd", "order_id"], aggfunc={"customer_id":pd.Series.nunique, "gmv_usd":np.sum, "order_id":pd.Series.nunique})
pivoted_table = pivoted_table[pivoted_table.index >= otro["submit_month"].min()].dropna(axis=1, how="all").copy()
assert pivoted_table["gmv_usd"].fillna(value=0).values.sum()/otro[otro["birthday_month"] >= otro["submit_month"].min()]["gmv_usd"].sum() - 1 < 0.0000004
pivoted_table.index = [str(x.year) + "-" + str(x.month) for x in pivoted_table.index]

print("-----------------#Clientes------------")
display((pivoted_table["customer_id"]).style.apply(highlight_max, axis=1).format("{:.1f}", na_rep="-"))

grouped_df = otro.groupby(["customer_id", "months_alive"]).agg({"birthday_month":pd.Series.unique, "category":pd.Series.nunique, "subcat":pd.Series.nunique}).reset_index()
pivoted_table_2 = pd.pivot_table(data=grouped_df, columns=["months_alive"], index=["birthday_month"], values=["category", "subcat"], aggfunc={"category":np.mean, "subcat":np.mean})
pivoted_table_2 = pivoted_table_2[pivoted_table_2.index >= otro["submit_month"].min()].dropna(axis=1, how="all").copy()
pivoted_table_2.index = [str(x.year) + "-" + str(x.month) for x in pivoted_table_2.index]